In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()
from pyspark.ml.regression import LinearRegression


In [3]:

data = spark.read.csv('../resources/data/Ecommerce_Customers.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [6]:
from pyspark.ml.linalg import Vectors   # linear algebra
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership','Yearly Amount Spent'],
    outputCol='features')

In [9]:
output=assembler.transform(data)

In [10]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [11]:
final_data = output.select('features','Yearly Amount Spent')

In [12]:
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [13]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                337|
|   mean| 498.03489359845395|
| stddev|  80.10910805641818|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [15]:
lr=LinearRegression(labelCol='Yearly Amount Spent')

In [16]:
lr_model = lr.fit(train_data)

In [17]:
test_result = lr_model.evaluate(test_data)

In [ ]:
test_result.residuals.show()    # residuals: difference between predicted value and actual labeled value

In [19]:
test_result.rootMeanSquaredError

5.402831379972191e-12

In [20]:
test_result.r2

1.0

In [21]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
# unlabeled_data = test_data.select('features')
unlabeled_data = final_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
+--------------------+
only showing top 5 rows



In [ ]:
prediction = lr_model.transform(unlabeled_data)
prediction.show()

In [ ]:
final_data.show()